In [32]:
from readii.negative_controls_refactor.negative_controls import (
    RandomizedControl,
    SampledControl,
    ShuffledControl
)
from readii.negative_controls_refactor.regions import (
    FullRegion,
    NonROIRegion,
    ROIRegion
)

from readii.negative_controls_refactor.manager import NegativeControlManager
from pathlib import Path
import pandas as pd
import SimpleITK as sitk
from rich import print as rprint
import numpy as np
from collections import OrderedDict
from radiomics import featureextractor, setVerbosity
setVerbosity(50)

In [2]:
# filepath setup
DATA_SOURCE = "TCIA"
DATASET_NAME = "NSCLC-Radiomics"
DATA_DIR = Path("../data/procdata", f"{DATA_SOURCE}_{DATASET_NAME}")
MIT_INDEX_DIR = "mit_" + DATASET_NAME
pyrad_output_path = DATA_DIR / "pyradiomics_features"

RANDOM_SEED = 10

In [3]:
def flatten_image(image: sitk.Image) -> sitk.Image:
    """Remove axes of image with size one. (ex. shape is [1, 100, 256, 256])

    Parameters
    ----------
    image : sitk.Image
        Image to remove axes with size one.

    Returns
    -------
    sitk.Image
        image with axes of length one removed.
    """
    imageArr = sitk.GetArrayFromImage(image)

    imageArr = np.squeeze(imageArr)

    return sitk.GetImageFromArray(imageArr)

In [4]:
manager = NegativeControlManager.from_strings(
    negative_control_types=["randomized", "shuffled", "sampled"],
    region_types=["roi", "non_roi", "full"],
    random_seed=RANDOM_SEED,
)

In [5]:
params = "../src/readii/data/shape_only_pyradiomics.yaml"
extractor = featureextractor.RadiomicsFeatureExtractor(params)

In [22]:
pyrad_dataset_df = pd.read_csv(DATA_DIR / f"pyradiomics_features/pyrad_{DATASET_NAME}_index.csv")

sample_row = pyrad_dataset_df.loc[0]

sample_image = flatten_image(sitk.ReadImage(sample_row.Image))
sample_mask = flatten_image(sitk.ReadImage(sample_row.Mask))

sample_feature_dir = pyrad_output_path / f"samples/{sample_row.ID}"
sample_feature_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
for neg_image, control_strategy_name, region_strategy_name in manager.apply(sample_image, sample_mask):
    rprint(
        f"Negative control image generated: {neg_image.GetSize()} using {control_strategy_name} and {region_strategy_name}"
    )
    
    try:
        sample_feature_vector = pd.Series(extractor.execute(neg_image, sample_mask))
    except Exception as e:
        print("Feature extraction failed: ", e)
    
    result_dict = sample_row.to_dict(into=OrderedDict)
    result_dict.update(sample_feature_vector)
    
    result_data_df = pd.DataFrame(data=result_dict)
    print(result_data_df.head())


Negative control image generated: (500, 500, 134) using randomized and roi

ValueError: Shape of passed values is (57, 1), indices imply (57, 57)

In [14]:
featurevector = extractor.execute(neg_image, sample_mask)

In [41]:
sample_dict

OrderedDict([('ID', 'NSCLC-Radiomics_001'),
             ('Image',
              '../data/procdata/TCIA_NSCLC-Radiomics/mit_NSCLC-Radiomics/NSCLC-Radiomics_001/CT_Series-82046/CT.nii.gz'),
             ('Mask',
              '../data/procdata/TCIA_NSCLC-Radiomics/mit_NSCLC-Radiomics/NSCLC-Radiomics_001/RTSTRUCT_Series-78236/GTV.nii.gz'),
             ('diagnostics_Versions_PyRadiomics', 'v3.1.4'),
             ('diagnostics_Versions_Numpy', '1.26.4'),
             ('diagnostics_Versions_SimpleITK', '2.4.1'),
             ('diagnostics_Versions_PyWavelet', '1.8.0'),
             ('diagnostics_Versions_Python', '3.12.10'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': False,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': [0.0, 0.0, 0.0],
               'interpolator': 'sitkBSpline',
               'preCrop':